In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score , accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge


In [19]:
df  = pd.read_csv('cleaned_data.csv')

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report , accuracy_score



In [21]:
X = df.drop("Heart Disease Status", axis=1)
y = df["Heart Disease Status"]

In [22]:
from sklearn.preprocessing import LabelEncoder

# Encode target column
le = LabelEncoder()
y = le.fit_transform(df["Heart Disease Status"])

In [23]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

# Optional: Replace missing categorical values randomly if needed
for col in categorical_features:
    if X[col].isnull().sum() > 0:
        categories = X[col].dropna().unique()
        np.random.seed(42)
        mask = X[col].isnull()
        X.loc[mask, col] = np.random.choice(categories, size=mask.sum(), replace=True)

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Combine transformers
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])


In [24]:
# Combine transformers
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])


In [25]:
# Define full pipeline with classifier
X = preprocessor.fit_transform(X)

In [26]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [27]:
from sklearn.metrics import accuracy_score

def evaluate_model(true, predicted):
    predicted = np.round(predicted)  # Convert regression output to class labels
    accuracy = accuracy_score(true, predicted)
    return accuracy


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier()
}


In [29]:
accuracy_list = []
model_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model_name = list(models.keys())[i]

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(model_name)
    model_list.append(model_name)

    print('Model Performance for Training Set')
    print("- Accuracy Score: {:.4f}".format(train_acc))

    print('-----------------------------')

    print('Model Performance for Test Set')
    print("- Accuracy Score: {:.4f}".format(test_acc))

    accuracy_list.append(test_acc)  

    print('=' * 35)
    print('\n')


Logistic Regression
Model Performance for Training Set
- Accuracy Score: 0.8003
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.7976


KNN
Model Performance for Training Set
- Accuracy Score: 0.8173
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.7680


Decision Tree
Model Performance for Training Set
- Accuracy Score: 1.0000
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.6629


Random Forest
Model Performance for Training Set
- Accuracy Score: 1.0000
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.7976


XGBoost
Model Performance for Training Set
- Accuracy Score: 0.9713
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.7843




c:\Users\vivek gupta\Desktop\Heart_disease_prediction\heart_venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:07:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


CatBoost
Model Performance for Training Set
- Accuracy Score: 0.8347
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.7976


AdaBoost
Model Performance for Training Set
- Accuracy Score: 0.8003
-----------------------------
Model Performance for Test Set
- Accuracy Score: 0.7976




In [30]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Split your data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
log_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier(n_estimators=250 , criterion='gini')
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
ada_clf = AdaBoostClassifier()

# Voting Classifier (Hard Voting or Soft Voting)
voting_clf = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('knn', knn_clf),
        ('dt', dt_clf),
        ('rf', rf_clf),
        ('xgb', xgb_clf),
        ('ada', ada_clf)
    ],
    voting='hard'  # change to 'soft' if models support predict_proba
)

# Fit ensemble model
voting_clf.fit(X_train, y_train)

# Predict
y_pred = voting_clf.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print("Voting Classifier Accuracy: {:.4f}".format(accuracy))


c:\Users\vivek gupta\Desktop\Heart_disease_prediction\heart_venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:07:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Voting Classifier Accuracy: 0.7976


In [31]:
# prediction throught only random forest

rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
accuracy

0.7975522692503825

In [32]:
y

array([0, 0, 0, ..., 1, 1, 1])